In [ ]:
# import dependencies
import os
import numpy as np
import pandas as pd
import scanpy as sc
import loompy as lp
import scipy.io as sio
from pyscenic.rss import regulon_specificity_scores
from pyscenic.plotting import plot_rss
import json
import zlib
import base64
from adjustText import adjust_text


import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as mcolors
import matplotlib.cm as cm


In [ ]:
#patch for numpy dependency issue in regulon_specificity_scores
np.float = float   

In [ ]:
sc.settings.njobs = 32
sc.settings.verbosity = 3 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.set_figure_params(dpi=150, fontsize=10, dpi_save=600)
sc.settings.figdir = NULL

In [ ]:
adata = sc.read_h5ad('/path/to/adata.h5ad')
adata

In [ ]:
nGenesDetectedPerCell = np.sum(adata.X>0, axis=1)
nGenesDetectedPerCell = pd.DataFrame(nGenesDetectedPerCell)
percentiles = nGenesDetectedPerCell.quantile([.01, .05, .10, .50, 1])
print(percentiles)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5), dpi=150)
sns.distplot(nGenesDetectedPerCell, norm_hist=False, kde=False, bins='fd')
for i,x in enumerate(percentiles.iloc[:,0]):
    fig.gca().axvline(x=x, ymin=0,ymax=1, color='red')
    ax.text(x=x, y=ax.get_ylim()[1], s=f'{int(x)} ({percentiles.index.values[i]*100}%)', color='red', rotation=30, size='x-small',rotation_mode='anchor' )
ax.set_xlabel('# of genes')
ax.set_ylabel('# of cells')
fig.tight_layout()

In [ ]:
lf = lp.connect('/path/to/pyscenic.loom', mode='r+', validate=False)
regulons = pd.DataFrame(lf.ra.Regulons)
auc_mtx = pd.DataFrame(lf.ca.RegulonsAUC, index=lf.ca.CellID)
meta = json.loads(zlib.decompress(base64.b64decode( lf.attrs.MetaData )))


In [ ]:
auc_mtx = auc_mtx.loc[adata.obs.index]

In [ ]:
regulons.to_csv("/output_path/regulons.csv")
sio.mmwrite("/output_path/regulonsAUC.mtx",auc_mtx)
auc_mtx.to_csv("/output_path/regulonsAUC.csv")

In [ ]:
# create a dictionary of regulons:
regulons = {}
for i,r in pd.DataFrame(lf.ra.Regulons,index=lf.ra.Gene).items():
    regulons[i] =  list(r[r==1].index.values)

In [ ]:
regulons

In [ ]:
cellAnnot = pd.concat(
    [
        pd.DataFrame( adata.obs['sample'], index=adata.obs.index ),
        pd.DataFrame( adata.obs['NMF_max'], index=adata.obs.index )
    ],
    axis=1
)
cellAnnot.columns = [
 'sample',
 'NMF_max'
]

In [ ]:
cellAnnot

In [ ]:
rss_cellType = regulon_specificity_scores(auc_mtx, cellAnnot['NMF_max'] )
rss_cellType

In [ ]:
cats = sorted(list(set(cellAnnot['NMF_max'])))

fig = plt.figure(figsize=(12, 18))
for c,num in zip(cats, range(1,len(cats)+1)):
    x=rss_cellType.T[c]
    ax = fig.add_subplot(2,3,num)
    plot_rss(rss_cellType, c, top_n=10, max_n=None, ax=ax)
    ax.set_ylim( x.min()-(x.max()-x.min())*0.05 , x.max()+(x.max()-x.min())*0.05 )
    for t in ax.texts:
        t.set_fontsize(12)
    ax.set_ylabel('')
    ax.set_xlabel('')
    adjust_text(ax.texts, autoalign='xy', ha='right', va='bottom', arrowprops=dict(arrowstyle='-',color='lightgrey'), precision=0.001 )
fig.text(0.5, 0.0, 'Regulon', ha='center', va='center', size='x-large')
fig.text(0.00, 0.5, 'Regulon specificity score (RSS)', ha='center', va='center', rotation='vertical', size='x-large')
plt.tight_layout()
plt.rcParams.update({
    'figure.autolayout': True,
        'figure.titlesize': 'large' ,
        'axes.labelsize': 'medium',
        'axes.titlesize':'large',
        'xtick.labelsize':'medium',
        'ytick.labelsize':'medium'
        })
plt.savefig("figures/BRO_malignant_RSS_top10.pdf", dpi=600, bbox_inches = "tight")
plt.show()